# Inicialización

En primer lugar instalamos y configuramos todas las dependencias de Spark para Python. De esta forma enlazaremos nuestro entorno con el servidor de Spark. Además configuraremos el entorno Spark con las variables que sean necesarias.



**NOTA:**

**La última versión de PySpark es la 3.5.4 [link](https://pypi.org/project/pyspark/#history)**

In [1]:
# Install spark-related dependencies
# !wget -q  https://apache.osuosl.org/spark/spark-3.5.4/spark-3.5.4-bin-hadoop3.tgz
# !tar xf spark-3.5.4-bin-hadoop3.tgz

!pip install -q findspark
!pip install pyspark
# Set up required environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "D:\CEIABD\spark-3.5.4-bin-hadoop3"

Vamos a verificar que el entorno Spark está bien creado

In [2]:
os.environ["SPARK_HOME"]

'D:\\CEIABD\\spark-3.5.4-bin-hadoop3'

Vamos a iniciar uan sesión de spark simple para testear nuestra instalación

1. Ejecutamos findspark.init() para hacer que pyspark sea importable como una biblioteca normal

In [3]:
echo %PATH%

c:\Users\alfre\AppData\Local\Programs\Python\Python311;c:\Users\alfre\AppData\Roaming\Python\Python311\Scripts;C:\Program Files\Common Files\Oracle\Java\javapath;C:\Program Files (x86)\Common Files\Oracle\Java\java8path;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0\;C:\Windows\System32\OpenSSH\;C:\Program Files\Java\jdk-17\bin;C:\Program Files\nodejs\;C:\Program Files\Git\cmd;C:\Program Files\WireGuard\;C:\Program Files\Amazon\AWSCLIV2\;C:\Program Files\Docker\Docker\resources\bin;C:\Program Files\Exiftool;C:\Users\alfre\AppData\Local\Programs\oh-my-posh\bin\;C:\Users\alfre\.cargo\bin;C:\Program Files\MySQL\MySQL Shell 8.0\bin\;C:\Users\alfre\AppData\Local\Programs\Python\Python311\Scripts\;C:\Users\alfre\AppData\Local\Programs\Python\Python311\;C:\Users\alfre\AppData\Local\Microsoft\WindowsApps;C:\Users\alfre\AppData\Local\Programs\Microsoft VS Code\bin;C:\msys64\usr\bin;C:\Pr

In [4]:
import findspark
findspark.init()

2. Crear un contexto Spark para ejecutar la aplicación

> NOTA: Un SparkContext representa la conexión al cluster de Spark, y puede utilizarse para crear RDDs y otros elementos. **Sólo puede haber un SparkContext activo**. Se debe detener (*stop()*) el SparkContext activo antes de crear uno nuevo.

In [5]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [7]:
conf = SparkConf()
conf.set("spark.ui.port", "4050")
conf.set("spark.appName", "Pi")
# create the context
sc = pyspark.SparkContext()
spark = SparkSession.builder.getOrCreate()
sc


KeyboardInterrupt: 

# Paralelizar el cálculo con parallelize:
Este método se utiliza para distribuir la colección de elementos del mismo tipo (datos u operaciones) para poder funcionar en paralelo.

sc.parallelize()

# Particiones en Spark
La partición significa que los datos completos no están presentes en un solo lugar. Se divide en varios fragmentos y estos fragmentos se colocan en diferentes nodos.

Si tiene una partición, Spark solo tendrá un paralelismo de una, incluso si tiene miles de ejecutores. Además, si tiene muchas particiones pero solo un ejecutor, Spark seguirá teniendo solo un paralelismo de uno porque solo hay un recurso de cálculo.

En Spark, las API de nivel inferior nos permiten definir la cantidad de particiones.

Tomemos un ejemplo sencillo para entender cómo la partición nos ayuda a dar resultados más rápidos. Crearemos una lista de 20 millones de números aleatorios entre 10 y 1000 y contaremos los números mayores a 200.

Veamos qué tan rápido podemos hacer esto con una sola partición:

In [ ]:
from random import randint

# create a list of random numbers between 10 to 1000
my_large_list = [randint(10,1000) for x in range(0,20000000)]

# create one partition of the list
my_large_list_one_partition = sc.parallelize(my_large_list,numSlices=1)

# check number of partitions
print(my_large_list_one_partition.getNumPartitions())
# >> 1

# filter numbers greater than equal to 200
my_large_list_one_partition = my_large_list_one_partition.filter(lambda x : x >= 200)

1


In [ ]:
# to calculate the time taken to execute the following command
%%time

# count the number of elements in filtered list
print(my_large_list_one_partition.count())
# >> 16162207

16166223
CPU times: user 78.6 ms, sys: 11.4 ms, total: 90 ms
Wall time: 13.2 s


Ahora, aumentemos el número de particiones a 5 y verifiquemos si obtenemos alguna mejora en el tiempo de ejecución:

In [ ]:
# create five partitions of the list
my_large_list_with_five_partition = sc.parallelize(my_large_list, numSlices=5)

# filter numbers greater than equal to 200
my_large_list_with_five_partition = my_large_list_with_five_partition.filter(lambda x : x >= 200)

In [ ]:
%%time

# count the number of elements in the filtered list
print(my_large_list_with_five_partition.count())

16166223
CPU times: user 57.5 ms, sys: 5.99 ms, total: 63.5 ms
Wall time: 9.29 s


Paramos el SparkContext

In [ ]:
sc.stop()